In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

In [9]:
#constant

data_raw_frame = 100
opacity = 0.5

In [10]:
def delta_t(radar_df, delta_t = 0.1):
    with open(f"data/raw data/radar_positions.json", "r") as file:
        radar_positions = {float(k): v for k, v in json.load(file).items()}

    for i, cords in radar_positions.items():
        for j, ax in enumerate(("X, (m)", "Y, (m)", "Z, (m)")):
            radar_df.loc[radar_df["radar_idx"] == i, ax] -= cords[j]

    vector_length = (radar_df['X, (m)']**2 + radar_df['Y, (m)']**2 + radar_df['Z, (m)'] ** 2)**0.5
    radar_df['RadialDelta'] = (delta_t - radar_df['(radar_point_ts - lidar_ts), (s)']) * radar_df['AbsoluteRadialVelocity']
    radar_df['XwithDelta'] = radar_df['X, (m)'] * (vector_length + radar_df['RadialDelta']) / vector_length
    radar_df['YwithDelta'] = radar_df['Y, (m)'] * (vector_length + radar_df['RadialDelta']) / vector_length
    radar_df['ZwithDelta'] = radar_df['Z, (m)'] * (vector_length + radar_df['RadialDelta']) / vector_length
    with open(f"data/raw data/radar_positions.json", "r") as file:
        radar_positions = {float(k): v for k, v in json.load(file).items()}

    for i, cords in radar_positions.items():
        for j, ax in enumerate(("XwithDelta", "YwithDelta", "ZwithDelta")):
            radar_df.loc[radar_df["radar_idx"] == i, ax] += cords[j]
    return radar_df


In [11]:
def normalize(radar_df):
    x_radar = radar_df[radar_df["QPDH0"] < 0.3]["X, (m)"]
    y_radar = radar_df[radar_df["QPDH0"] < 0.3]["Y, (m)"]
    return x_radar, y_radar

In [15]:
for frame_num in tqdm(range(0, data_raw_frame)):
    frame_radar = pd.read_csv(f"data/processed data/radar_data_{frame_num}.csv")
    frame_lidar = pd.read_csv(f"data/processed data/lidar_data_{frame_num}.csv")

    plt.figure(figsize = (11, 5))
    plt.xlim([-110, 110])
    plt.ylim([-50, 50])
    
    frame_size = len(frame_lidar["r, (reflectance)"])

    gradient = []
    for i in range(frame_size):
        color = (frame_lidar["r, (reflectance)"][i] / 255) ** 0.3
        gradient.append((color, 0, 1 - color, opacity))
    
    plt.scatter(frame_lidar["X, (m)"], frame_lidar["Y, (m)"], s=1, c=gradient)
    frame_radar_normalize = normalize(frame_radar)
    plt.scatter(frame_radar_normalize[0], frame_radar_normalize[1], s=1, c = "red")

    plt.title(f"Frame {frame_num}")
    plt.savefig(f"data/radar_lidar_png/Frame_{frame_num}.png")
    plt.close()

100%|██████████| 100/100 [1:06:51<00:00, 40.12s/it] 
